In [ ]:
#first install the hypercoast
%pip install "hypercoast[extra]"

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import hypercoast
from hypercoast import Map

In [56]:
#Download the csv file I gathered from the dashboard
#the codes for downloading the csv files will be in another notebook
difcb=pd.read_csv("https://github.com/oceanhackweek/ohw24_proj_pace_us/raw/main/contributor_folders/Gulce/mvco_difcb_data.csv")
difcb.dtypes

date                 object
max_value_column     object
count                 int64
latitude            float64
longitude           float64
genus                object
dtype: object

In [17]:
# Step 2: Convert the cleaned date strings to datetime objects
difcb['date'] = pd.to_datetime(difcb['date'], format='%Y%m%d')
difcb.head()

,date,max_value_column,count,latitude,longitude
0,2024-07-01,Acantharia,1,41.362,-70.5226
1,2024-07-01,Acanthoica_quattrospina,3739,41.362,-70.5226
2,2024-07-01,Alexandrium_catenella,10,41.362,-70.5226
3,2024-07-01,Amoeba,1974,41.362,-70.5226
4,2024-07-01,Amphidinium,285,41.362,-70.5226


Now that i arranged my ifcb data from the dashboard for mvco,<br>
 I can start downloading the PACE chl-a  with hypercoast

In [50]:
import hypercoast

# Step 1: Login to NASA Earthdata
hypercoast.nasa_earth_login()


In [51]:
# Step 2: Search for Chl-a data within a specific temporal range
temporal = ("2024-06-01", "2024-07-21")  # Adjust date range as needed
results = hypercoast.search_pace_chla(temporal=temporal)

# Step 3: Download the Chl-a data
hypercoast.download_nasa_data(results, "chla")

# Print the search results to see what you found
print(results)


QUEUEING TASKS | :   0%|          | 0/51 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/51 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/51 [00:00<?, ?it/s]

[Collection: {'ShortName': 'PACE_OCI_L3M_CHL_NRT', 'Version': '2.0'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'EastBoundingCoordinate': 180, 'SouthBoundingCoordinate': -90, 'WestBoundingCoordinate': -180, 'NorthBoundingCoordinate': 90}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2024-06-01T00:00:00Z', 'EndingDateTime': '2024-06-01T23:59:59Z'}}
Size(MB): 2.7839736938476562
Data: ['https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240601.L3m.DAY.CHL.V2_0.chlor_a.0p1deg.NRT.nc'], Collection: {'Version': '2.0', 'ShortName': 'PACE_OCI_L3M_CHL_NRT'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -180, 'SouthBoundingCoordinate': -90, 'EastBoundingCoordinate': 180, 'NorthBoundingCoordinate': 90}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2024-06-02T00:00:00Z', 'EndingDateTime': '2024-06-02T23:59:59Z'}}
Size(MB): 2.72006988

In [52]:
from hypercoast import read_pace_chla, pace_chla_to_image

# Step 4: Read the downloaded data files into an array
files = "chla/*.nc"  # Adjust file path to where your data is saved
array = read_pace_chla(files)

# Step 5: Select data for a specific date range and calculate the average
start_date = "2024-07-01"
end_date = "2024-07-17"
selected_array = array.sel(date=slice(start_date, end_date))
average_array = selected_array.mean(dim='date')

# Step 6: Convert the averaged array to an image format
single_image = pace_chla_to_image(average_array)


In [53]:
import pandas as pd


# Extract the genus from the species name
difcb['genus'] = difcb['max_value_column'].str.split('_').str[0]

# Aggregate the counts by genus and date, and filter out counts less than 50
aggregated_data = (
    difcb.groupby(['date', 'genus'], as_index=False)
    .agg({'count': 'sum', 'latitude': 'first', 'longitude': 'first'})
    .query('count >= 100')
)


In [54]:
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import ipywidgets as widgets
from hypercoast import Map

# Function to create species plot for a given date and location with log-transformed Y-axis
def create_genus_plot(species_data):
    fig, ax = plt.subplots(figsize=(15, 10))
    ax.bar(species_data['genus'], species_data['count'])
    ax.set_yscale('log')
    ax.set_title(f"Genus Distribution (Log Scale)", fontsize=20)
    ax.set_xlabel('Genus', fontsize=18)
    ax.set_ylabel('Count (log scale)', fontsize=18)
    ax.tick_params(axis='x', labelrotation=90, labelsize=12)

    buf = BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    html_img = f'<img src="data:image/png;base64,{img_base64}" style="width:100%;">'
    return html_img

# Group data by location
locations = aggregated_data.groupby(['latitude', 'longitude'])

# Step 7: Create the map centered on your stations
m = Map(center=[41.362, -70.5226], zoom=8)
m.add_basemap("Hybrid")

# Step 8: Add the averaged Chl-a raster layer to the map
m.add_raster(
    single_image,
    cmap="jet",
    vmin=-1,
    vmax=2,
    layer_name="Chlorophyll a",
    zoom_to_layer=False,
)

# Step 9: Add species distribution markers to the map
for (lat, lon), location_data in locations:
    
    # Generate a time series plot or dropdown for each location
    date_options = location_data['date'].unique()
    date_to_html = {}
    
    for date in date_options:
        genus_data = location_data[location_data['date'] == date]
        date_to_html[date] = create_genus_plot(genus_data)
    
    # Construct the HTML for the popup
    popup_html = "<strong>Station Info</strong><br>"
    popup_html += "Location: {:.6f}, {:.6f}<br>".format(lat, lon)
    popup_html += "Select Date:<br>"
    
    # Add a dropdown for date selection
    popup_html += '<select id="date_selector" onchange="updateImage(this.value)">'
    for date in date_options:
        popup_html += f'<option value="{date}">{date}</option>'
    popup_html += '</select><br>'
    
    # Add the initial genus distribution image
    first_date = date_options[0]
    popup_html += f'<div id="species_img">{date_to_html[first_date]}</div>'
    
    # Add script to update image on date change
    popup_html += '''
    <script>
    function updateImage(date) {
        const images = {};
        '''
    # Embed each image in the JavaScript
    for date, img_html in date_to_html.items():
        popup_html += f'images["{date}"] = `{img_html}`;\n'
    
    popup_html += '''
        document.getElementById("species_img").innerHTML = images[date];
    }
    </script>
    '''
    
    # Create an HTML widget from the string and set the size of the popup
    popup_widget = widgets.HTML(value=popup_html)
    popup_container = widgets.Box([popup_widget], layout=widgets.Layout(width='600px', height='500px'))
    
    # Add the point to the map
    location = (lat, lon)
    m.add_marker(location=location, popup=popup_container)

# Step 10: Display the map with Chl-a raster and species markers
m


Map(center=[41.362, -70.5226], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…